In [1]:
#takes 
from __future__ import division, absolute_import, print_function
%load_ext autoreload
%autoreload 2
import os
print(os.getcwd())
import hepran
import hepran.bzipscore as bz
from hepran.bzipscore import *
import hepran.utils as u
import hepran.registers as r
import numpy as np
CCO = os.environ.get("CCO",r"C:\Projekti_KI\ortoCC\CoiledCoilOrtho")
from glob import glob
import pandas as pd
from StringIO import StringIO
import collections
from gather_utils import *

import pandas as pd

d:\data\ortoCC\fine_tune_scoring_function\OUT_SETS
HERE
['KNLK', 'ENLE', 'EILE', 'EILE']


In [2]:
sets = glob("!02_OUT_bcf/*.set")  
sets += glob("!03_OUT_additional/*.set")  

final_fasta = collections.OrderedDict()
final_pairs = []

In [3]:
for _set in sets:
    set_data = u.load_set(_set)
    fasta_data = u.load_fasta(u.replace_extension(_set,'fasta'))
    
    #change first IDs to D
    #fasta_data = u.change_nth_res(fasta_data, pos=0, res='D')
    #ids are loaded from set. Important, because fasta can then be larger...
    set_ids = bz.get_ids_from_pairs(set_data)
    all_pairs = u.make_all_pairs(set_ids, is_symmetric=True)
    u.append_pairs(final_pairs, all_pairs, is_symmetric=True)
    u.append_fasta(final_fasta, fasta_data)
    

print("pairs:",len(final_pairs))
print("IDs:"  ,len(final_fasta))

u.save_dict_to_fasta(final_fasta, "!all-sym.fasta")
u.save_set(final_pairs, "!all-sym.pairs")

pairs: 8249
IDs: 448


In [4]:
## Load the pairs for which we have experimental data

seqs_df = pd.read_excel("!04_OUT_additional_pairs/reliable-seq+Tm.xlsx", sheet_name="sequence")
#throw out whitespace and -
#warning - alignment is lost!
seqs_df.drop(0, inplace=True)
seqs_df.sequence = seqs_df.sequence.str.replace(" ", "")
seqs_df.sequence = seqs_df.sequence.str.replace("-", "")
#Limit to 
seqs_df.sequence = seqs_df.sequence.str[0:29]
seqs_exp = collections.OrderedDict()
for i, row in seqs_df.iterrows():
    seqs_exp[row.peptide] = row.sequence
    
seqs_exp;

pairs_df = pd.read_excel("!04_OUT_additional_pairs/reliable-seq+Tm.xlsx", sheet_name="Tm")
pairs_df
pairs_exp = [(row.peptide1, row.peptide2) for i, row in pairs_df.iterrows()]
pairs_exp;

##append the pairs
u.append_pairs(final_pairs, pairs_exp, is_symmetric=True)
u.append_fasta(final_fasta, seqs_exp)
    

print("pairs:",len(final_pairs))
print("IDs:"  ,len(final_fasta))



pairs: 8273
IDs: 483


In [5]:
cross_pairs = glob("!05_cross_pairs/*.pairs")
for cp in cross_pairs:
    print(cp)
    set_data = u.load_set(cp)
    fasta_data = u.load_fasta(u.replace_extension(cp,'fasta'))
    
    u.append_pairs(final_pairs, set_data, is_symmetric=True)
    u.append_fasta(final_fasta, fasta_data)

!05_cross_pairs\aa_ge_crosspairs.pairs
!05_cross_pairs\NIEK_crosspairs.pairs


In [6]:
aliases = get_aliases(final_fasta)
u.ensure_dir("!99_OUT_pairs")
#aliases.to_csv("!99_OUT_pairs/aliases.csv")
aliases.index

Index([u'DEIAALEAEIAALEAEIAALEAEIAALEG', u'DEIAALEAEIAALEAENAALEAEIAALEG',
       u'DEIAALEAEIAALEAENAALEAENAALEG', u'DEIAALEAEIAALEAKIAALKAENAALEG',
       u'DEIAALEAEIAALEAKIAALKAKNAALKG', u'DEIAALEAEIAALEAKNAALKAKNAALKG',
       u'DEIAALEAENAALEAEIAALEAKIAALKG', u'DEIAALEAENAALEAEIAALEAKNAALKG',
       u'DEIAALEAENAALEAENAALEAEIAALEG', u'DEIAALEAENAALEAKIAALKAENAALEG',
       u'DEIAALEAENAALEAKIAALKAKIAALKG', u'DEIAALEAENAALEAKNAALKAKIAALKG',
       u'DEIAALEAKIAALKAENAALEAEIAALEG', u'DEIAALEAKIAALKAENAALEAENAALEG',
       u'DEIAALEAKIAALKAENAALEAKIAALKG', u'DEIAALEAKIAALKAKNAALKAEIAALEG',
       u'DEIAALEAKIAALKAKNAALKAKNAALKG', u'DEIAALEAKNAALKAEIAALEAENAALEG',
       u'DEIAALEAKNAALKAEIAALEAKIAALKG', u'DEIAALEAKNAALKAEIAALEAKNAALKG',
       u'DEIAALEAKNAALKAENAALEAEIAALEG', u'DEIAALEAKNAALKAKNAALKAEIAALEG',
       u'DEIAALEAKNAALKAKNAALKAKIAALKG', u'DEIQQLEEEISQLEQKNSQLKEKNQQLKY',
       u'DENAALEAEIAALEAEIAALEAKNAALKG', u'DENAALEAEIAALEAENAALEAEIAALEG',
       u'DENAALEAEIAALEAE

In [7]:

def save_pairs(pairs, fasta, out_name):
    u.save_dict_to_fasta(fasta, u.ensure_extension(out_name, ".fasta"))
    u.save_set(pairs, u.ensure_extension(out_name, ".pairs"))
    seqs_dict = [dict(ID1=fp1, ID2=fp2, 
                      seq1=fasta[fp1].replace("-","").replace(" ",""), 
                      seq2=fasta[fp2].replace("-","").replace(" ",""))  for fp1, fp2 in pairs]
    df = pd.DataFrame.from_dict(seqs_dict)
    df.to_csv(u.ensure_extension(out_name, ".pairs_seq"))  

In [14]:
alias_dict = get_alias_dict(aliases)
import json
with open('!99_OUT_pairs/aliases.dict.json', 'w') as file:
     file.write(json.dumps(alias_dict, indent=4)) 

In [15]:
zipped_fasta = zip_fasta(final_fasta, alias_dict) 
print(len(final_fasta),len(zipped_fasta))
print(len(final_fasta)-len(zipped_fasta))
zipped_pairs = zip_pairs(final_pairs, alias_dict)

931 801
130


In [16]:
assert u.check_pairs_and_fasta(final_fasta, final_pairs)
print("pairs:",len(final_pairs))
print("IDs:"  ,len(final_fasta))

pairs: 9553
IDs: 931


In [17]:
assert u.check_pairs_and_fasta(zipped_fasta, zipped_pairs)
print("pairs:",len(zipped_pairs))
print("IDs:"  ,len(zipped_fasta))
print("Diff:" ,len(final_pairs)-len(zipped_pairs))

pairs: 9462
IDs: 801
Diff: 91


In [18]:
    
flipped_final_pairs = flip_pairs(final_pairs)
assert u.check_pairs_and_fasta(final_fasta, flipped_final_pairs)
print("pairs:",len(flipped_final_pairs))
print("IDs:"  ,len(flipped_final_pairs))
print("Diff:" ,len(flipped_final_pairs)-len(final_pairs)*2) 
save_pairs(flipped_final_pairs, final_fasta, "!99_OUT_pairs\\flipped_final_pairs")

pairs: 18653
IDs: 18653
Diff: -453


In [19]:

    
flipped_zipped_pairs = flip_pairs(zipped_pairs)
assert u.check_pairs_and_fasta(final_fasta, flipped_zipped_pairs)
print("pairs:",len(flipped_zipped_pairs))
print("IDs:"  ,len(flipped_zipped_pairs))
print("Diff:" ,len(flipped_zipped_pairs)-len(flipped_final_pairs))    
save_pairs(flipped_zipped_pairs, final_fasta, "!99_OUT_pairs\\flipped_zipped_final_pairs")

pairs: 18491
IDs: 18491
Diff: -162


In [21]:
#check zipping
for p1, p2 in flipped_final_pairs:
    #print(p1,p2)
    a1, a2 = make_aliased_pair((p1,p2), alias_dict, sort=False)
    if not final_fasta[p1]==final_fasta[a1]:
        print(p1, a1, final_fasta[p1], final_fasta[a1])
    if not final_fasta[p2]==final_fasta[a2]:
        print(p2, a2, final_fasta[p2], final_fasta[a2])

GCN4-S2t GCN2_t QLEDKVEELLRKNYHLENEVERLKKLVG ----QLEDKVEELLRKNYHLENEVERLKKLVG----
GCN4-S2t GCN2_t QLEDKVEELLRKNYHLENEVERLKKLVG ----QLEDKVEELLRKNYHLENEVERLKKLVG----


In [22]:
  #check zipping
for p1, p2 in flipped_final_pairs:
    #print(p1,p2)
    a1, a2 = make_aliased_pair((p1,p2), alias_dict, sort=False)
    if (a1, a2) not in flipped_zipped_pairs:
        print ((p1,p2))
        print ((a1,a2))

In [23]:
cp = "4H1046-aA, 4H2579-aA, 4H3961-aA P5-aA P6-aA".split()

In [30]:
for p1, p2 in flipped_final_pairs:
    if (p1 in cp) or (p2 in cp):
        print(p1, "\t", p2,"\t", final_fasta[p1], final_fasta[p2])
        a1, a2 = make_aliased_pair((p1, p2), alias_dict=alias_dict, sort=False)
        assert (a1, a2) in flipped_zipped_pairs
        print(a1, "\t", a2,"\t", final_fasta[a1], final_fasta[a2])
        print()
        
        assert final_fasta[p1]==final_fasta[a1]
        assert final_fasta[p2]==final_fasta[a2]

AP1-aA 	 P5-aA 	 DELAAIEAELAAEEAENAALEAEAAALEG DENAALEAKIAALKAKNAALKAEIAALEG
AP1-aA 	 4H1401-aA 	 DELAAIEAELAAEEAENAALEAEAAALEG DENAALEAKIAALKAKNAALKAEIAALEG

P5-aA 	 AP1-aA 	 DENAALEAKIAALKAKNAALKAEIAALEG DELAAIEAELAAEEAENAALEAEAAALEG
4H1401-aA 	 AP1-aA 	 DENAALEAKIAALKAKNAALKAEIAALEG DELAAIEAELAAEEAENAALEAEAAALEG

AP2-aA 	 P5-aA 	 DKLAAIKAKLAANKAKKAALKAKAAALKG DENAALEAKIAALKAKNAALKAEIAALEG
AP2-aA 	 4H1401-aA 	 DKLAAIKAKLAANKAKKAALKAKAAALKG DENAALEAKIAALKAKNAALKAEIAALEG

P5-aA 	 AP2-aA 	 DENAALEAKIAALKAKNAALKAEIAALEG DKLAAIKAKLAANKAKKAALKAKAAALKG
4H1401-aA 	 AP2-aA 	 DENAALEAKIAALKAKNAALKAEIAALEG DKLAAIKAKLAANKAKKAALKAKAAALKG

AP3-aA 	 P5-aA 	 DELAAIEAEIAALEAKKAALKAKNAALKG DENAALEAKIAALKAKNAALKAEIAALEG
AP3-aA 	 4H1401-aA 	 DELAAIEAEIAALEAKKAALKAKNAALKG DENAALEAKIAALKAKNAALKAEIAALEG

P5-aA 	 AP3-aA 	 DENAALEAKIAALKAKNAALKAEIAALEG DELAAIEAEIAALEAKKAALKAKNAALKG
4H1401-aA 	 AP3-aA 	 DENAALEAKIAALKAKNAALKAEIAALEG DELAAIEAEIAALEAKKAALKAKNAALKG

AP4-aA 	 P5-aA 	 DELAANEAELAAEEAKLAAAKAKAAALKG

In [9]:
l

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [10]:
l[0]

0

In [11]:
l[1:
]

[1, 2, 3, 4, 5, 6, 7, 8, 9]